# Connect to ClickHouse Database

In [146]:
!mkdir -p /home/jupyter/datasphere/project/.clickhouse && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
     --output-document /home/jupyter/datasphere/project/.clickhouse/root.crt && \
chmod 0600 /home/jupyter/datasphere/project/.clickhouse/root.crt

--2024-09-12 11:21:43--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’

/home/jupyter/datas 100%[===================>]   3.50K  --.-KB/s    in 0s      

2024-09-12 11:21:44 (1.66 GB/s) - ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’ saved [3579/3579]



In [147]:
# указываем параметры подключения
CA = '/home/jupyter/datasphere/project/.clickhouse/root.crt' # если вы не меняли путь при скачивании сертификата

VERIFY = True
SECURE = True
USER = 'admin'
PASSWORD = 'hakaton1'
HOST = "rc1d-uq593a42e02ikh2k.mdb.yandexcloud.net" # адрес Clickhouse
PORT = 8443

In [148]:
!pip install clickhouse_connect > /dev/null

In [149]:
# импортируем библиотеку
import clickhouse_connect

# выполняем запрос - в случае успешного выполнения будет показана текущая версия Clickhouse
with clickhouse_connect.get_client(
            host=HOST, port=PORT, username=USER,
            password=PASSWORD, secure=SECURE, verify=VERIFY, ca_cert=CA) as ch_client:
        print(ch_client.command('SELECT version()'))

24.3.10.33


# Генерация IAM токена

In [150]:
# Устанавливаем пакеты, необходимые для обращения к YandexGPT API
%pip uninstall jwt
%pip install PyJWT -U

In [151]:
import requests
import json
import time
import jwt
import os

In [152]:
# Замените <идентификатор_сервисного_аккаунта> на ваше значение
service_account_id = "aje286em9u4ibs8572di"

In [153]:
with open('/content/authorized_key.json') as f:
    authorized_key = json.load(f)

In [154]:
# Замените <идентификатор_ключа> на ваше значение
key_id = "ajeo9id7551hpq15fv3o"
private_key = authorized_key['private_key']

In [155]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
iam_token = x['iamToken']

# RAG Pipeline

In [156]:
!pip install -q langchain_community langchain yandexcloud > /dev/null

In [157]:
# llm
folder = 'b1g01stbeeo96j8kfaiv'
model_uri = f'gpt://{folder}/yandexgpt/latest'
model_uri

'gpt://b1g01stbeeo96j8kfaiv/yandexgpt/latest'

In [158]:
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

# Cоздаем объект YandexGPTEmbeddings для построения векторов с помощью YandexGPT
embeddings = YandexGPTEmbeddings(iam_token=iam_token, folder_id=folder)

In [159]:
from langchain_community.vectorstores import Clickhouse, ClickhouseSettings

ch_config = ClickhouseSettings(host=HOST, port=PORT, username=USER, password=PASSWORD)

docsearch = Clickhouse(embedding=embeddings, config=ch_config, verify=VERIFY, ca_cert=CA)

In [160]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 5}
)

In [161]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """
Ты - генератор интерфейсов по запросу пользователя на естественном языке.
Твой ответ должен содержать только код и не должен содержать описания или другого постороннего текста.
Ты обязан использовать дизайн компоненты системы НЛМК.
Учитывай, что твой вывод должен быть валидным React.js кодом, который можно запускать без ошибок.
Ответь на вопрос, основываясь только на следующем контексте:
{context}

Вопрос: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nТы - генератор интерфейсов по запросу пользователя на естественном языке.\nТвой ответ должен содержать только код и не должен содержать описания или другого постороннего текста.\nТы обязан использовать дизайн компоненты системы НЛМК.\nУчитывай, что твой вывод должен быть валидным React.js кодом, который можно запускать без ошибок.\nОтветь на вопрос, основываясь только на следующем контексте:\n{context}\n\nВопрос: {question}\n'))])

In [162]:
from langchain_community.llms import YandexGPT

llm = YandexGPT(iam_token=iam_token, model_uri=model_uri, tempreture=0.6)

In [163]:
# Chain
chain = prompt | llm

In [164]:
question = "Создай страницу c Sidebar слева, Header сверху страницы, далее ниже по горизонтале 9 шт. Input, справа от Input по горизонтале будет 2 шт. Button, далее ниже Tabs, далее ниже по вертикале 11 шт. Input, внизу страницы будет 2 шт. Button."

# Decomposition

In [165]:
stupid = YandexGPT(iam_token=iam_token, model_uri=model_uri, tempreture=0.35)

In [166]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """
Ты ассистент, который разделяет входящий запрос на несколько подзапросов.
В основном входящий запрос будет требовать сгенерировать веб-интерфейс по описанию.
Разбей запрос на набор подзадач генерации отдельных высокоуровневых веб-компонент.
Каждая подзадача должна соответствовать одной единственной веб-компоненте.
Если в запросе указано название компоненты, то обязательно сохрани его.
Если в запросе указано относительное положение компненты, то обязательно сохрани его
и продублируй название компоненты, относительно которой она будет располагаться, в
соотвествующем подзапросе.
В начале добавь подзапрос - "Позиционируй все элементы при помощи компоненты Grid".
Сгенерируй несколько подзапросов для запроса: {question}.

Обязательно соблюдай правила форматирования ответа:
- Ничего кроме самих подзапросов писать не нужно.
- Один подзапрос - одно предложение.
- Каждый подзапрос должен быть написан в отдельной строке.

Пример:
Запрос пользователя - создай кнопку, слевой от неё текст люблю щенят
Твой ответ:
1. Создай первый элемент кнопка 'Button'
2. С помощью компоненты 'Grid' добавь слева от первого элемента (кнопки) второй элемент Input c тестом "люблю щенят"
"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [167]:
from langchain_core.output_parsers import StrOutputParser

def parse_final_answer(output: str) -> list[str]:
    str_list = list(filter(None, output.split("\n")))
    return str_list

# Chain
generate_queries_decomposition = ( prompt_decomposition | stupid | StrOutputParser() )

# Run
questions = generate_queries_decomposition.invoke({"question": question})

In [168]:
[questions]

['Позиционируй все элементы при помощи компоненты Grid.\n\n1. Создай компонент Sidebar слева от основного содержимого страницы.\n2. Добавь компонент Header сверху страницы.\n3. Ниже по горизонтали размести 9 элементов Input.\n4. Справа от каждого элемента Input по горизонтали разместите 2 элемента Button.\n5. Ниже разместите компонент Tabs.\n6. Ещё ниже по вертикали разместите 11 элементов Input.\n7. Внизу страницы разместите 2 элемента Button.']

In [169]:
# Prompt
template = """
Ты - генератор веб-интерфейсов по запросу пользователя на естественном языке.
Твой ответ должен содержать только код и не должен содержать описания или другого постороннего текста.
Ты обязан использовать дизайн компоненты системы НЛМК.
Вывод должен быть валидным React.js кодом, который запускается без ошибок.
Проверяй, чтобы в импортируемых компонентах не было повторений.

К каждой компоненте НЛМК добавь префикс "DS" c точкой на конце. Например: DS.Sidebar, DS.Input, DS.Button, DS.Box и т. д.
Обязательно сохраняй вложенность тегов. Например, Tabs.Tab должен в итоге быть DS.Tabs.Tab.

Не ставь многоточие (три точки - "..."), чтоб пропустить элементы.
Каждый элемент ОБЯЗАТЕЛЬНО должен быть прописан в выводе.
Также НЕ используй комментарии с многоточием (три точки "...").
Например, если написано "Добавить 9 кнопок", значит все 9 элементов должны быть в выводе.

Обращай внимание на объявления переменных в дополнительном контексте (переменные имеют модифкатор const).
Например, если компонент использует переменные, как в коде ниже (DS.Tab использует activeTab):
```javascript
...
<>
  <DS.Tabs>
    <DS.Tabs.Tab
      label="Входящие"
      active={{0 === Number(activeTab)}}
      onClick={{() => setActiveTab(0)}}
    />
  </DS.Tabs>
</>
...
```
То ОБЯЗАТЕЛЬНО вставь объявление activeTab вместе с функцией setActiveTab выше:
```javascript
...
const [activeTab, setActiveTab] = useState(0);

return (
  <>
    <DS.Tabs>
      <DS.Tabs.Tab
        label="Входящие"
        active={{0 === Number(activeTab)}}
        onClick={{() => setActiveTab(0)}}
      />
    </DS.Tabs>
  </>
)
...
```

Ответь на вопрос, основываясь на коде из дополнительного контекста.

Бери за основу код из дополнительного контекста и накладывай изменения на него.
Дополнительный контекст, относящийся к запросу:

\n --- \n {context} \n --- \n

Запросы, на которые нужно ответить:
\n --- \n {question} \n --- \n

Обязательно сохрани код, указанный ниже.
Код:
\n --- \n {q_a_pairs} \n --- \n
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [170]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""

    formatted_string = ""
    # formatted_string += f"Запрос:\n{question}\n\nОтвет:\n{answer}\n\n"
    formatting_string = f"{answer}\n\n"

    return formatted_string.strip()

def show(docs):
  print("retriever output")
  print('-' * 100)
  for doc in docs:
    print(doc)
  print('-'* 100)
  return docs

q_a_pairs = """
import { useState, useEffect } from "react";
import * as DS from "@nlmk/ds-2.0";

const App = () => {
    {/* Генерируй код внутри */}
};

export default App;
"""

for q in [questions]:

    print(q_a_pairs)

    rag_chain = (
    {"context": itemgetter("question") | retriever | show,
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")}
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    # q_a_pairs = q_a_pairs + '\n---\n' + q_a_pair
    q_a_pairs = q_a_pair

    # print(answer)


import { useState, useEffect } from "react";
import * as DS from "@nlmk/ds-2.0";

const App = () => {
    {/* Генерируй код внутри */}
};

export default App;

retriever output
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------


In [145]:
print(answer)

```
import { useState, useEffect } from "react";
import * as DS from "@nlmk/ds-2.0";

const App = () => {
  const [activeTab, setActiveTab] = useState(0);
  return (
    <>
      <DS.Sidebar />
      <DS.Grid borderRadius="var(--4-border)">
        <DS.Header />
        {/* Позиционируй все элементы при помощи компоненты Grid */}
        <DS.Tabs>
          <DS.Tabs.Tab
            label="Входящие"
            active={0 === Number(activeTab)}
            onClick={() => setActiveTab(0)}
          />
        </DS.Tabs>
        <div style={{ display: "flex", flexDirection: "row" }}>
          {/* Создай компонент Sidebar слева от компонента Grid */}
          <DS.Input />
          {/* Добавь ниже по горизонтали девять компонентов Input */}
          {Array.from({ length: 9 }, (_, i) => (
            <DS.Input key={i} />
          ))}
          {/* Расположи два компонента Button справа от каждого из девяти компонентов Input, находящихся ниже по горизонтали */}
          {Array.from(
    

# Использовать релевантные для запроса документы

In [ ]:
results = docsearch.similarity_search(query=question, k=1)
print(len(results))
for doc in results: print(f"* {doc.page_content} [{doc.metadata}]")

1
* Card
Компонент Card является функциональным компонентом, который предназначен для отображения карточки с различными элементами интерфейса, такими как изображение, заголовок, описание, значки, селектор и группа кнопок.
Card по умолчанию
import { Button, Card, Typography, Select } from '@nlmk/ds-2.0';

export default  App = () => (
  <>
    <div style={{
      backgroundColor: 'var(--steel-20)',
      width: 'fit-content',
      padding: '20px'
  }}>
    <Card>
      <div
        style={{
          height: '200px',
          width: '300px',
          display: 'flex',
          flexDirection: 'column',
          gap: '10px'
        }}
      >
        <Typography variant='Heading3'>
          Заголовок
        </Typography>

        <Select
          label="Выберите язык программирования"
          options={[
            {
              label: 'C++',
              value: 'C++'
            },
            {
              label: 'C#',
              value: 'C#'
            }]}
        />



In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke(question)
print(response)

```
import { Button, Card, Typography } from '@nlmk/ds-2.0';

export default function App() {
  return (
    <div style={{ backgroundColor: 'var(--steel-20)', width: 'fit-content', padding: '20px' }}>
      <Card>
        <img src="product-image.jpg" alt="Product image" width="100%" height="150px" />
        <div style={{ height: '200px', width: '300px', display: 'flex', flexDirection: 'column', gap: '10px' }} >
          <Typography variant='Heading3'>
            Название продукта
          </Typography>
          <Select label="Выберите язык программирования" options={[{ label: 'C++', value: 'C++' }, { label: 'C#', value: 'C#' }]} />
          <div style={{ display: 'flex', gap: '8px', marginTop: 'auto' }} >
            <Button>Купить</Button>
            <Button variant="secondary">Подробнее</Button>
          </div>
        </div>
      </Card>
    </div>
  );
}
```
